<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Algoritmo%20cu%C3%A1ntico%20de%20optimizaci%C3%B3n%20aproximada%20(QAOA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos qiskit en el notebook
!pip install qiskit qiskit-aer qiskit-algorithms qiskit-optimization qiskit_machine_learning qiskit-ibm-runtime pylatexenc

In [ ]:
import numpy as np
from math import sqrt

# importing Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator, StatevectorSimulator

# import basic plot tools
from qiskit.visualization import plot_histogram

# Funciones auxiliares

# Función para obtener y mostrar el vector de estado
def obten_estado(qcirc, etiqueta="|\psi\!\!> = ", bloch=False):
  estado = Statevector.from_instruction(qcirc)
  display(estado.draw('latex', prefix=etiqueta, max_size=2**qc.num_qubits))
  if bloch:
    display(estado.draw('bloch'))

# **Algoritmo cuántico de optimización aproximada (QAOA)**

### Contenidos

1. [Discretización de la evolución adiabática (*Trotterization*)](#trotter)
1. [Quantum Approximate Optimization Algorithm (QAOA)](#qaoa)
1. [Programas cuadráticos con restricciones cuadráticas](#cuadraticos)
1. [MAX-CUT como QUBO](#maxcut)

<a name="trotter"></a>
## **Discretización de la evolución adiabática (*Trotterization*)**

Podemos replicar la evolución adiabática en un computador cuántico basado en puertas discretizando la evolución continua del Hamiltoniano.

La evolución temporal de un sistema viene dada por la ecuación de Schrödinger:

$$i\hbar \frac{d}{dt}|\psi(t)\rangle = H(t)|\psi(t)\rangle$$

En general, $H(t)$ depende del tiempo. Pero en un intervalo temporal $[t_i, t_i+\Delta t]$ suficientemente pequeño, podemos considerar $H(t_i)$ constante, verificándose que:

$$|\psi(t_i+\Delta t)\rangle = e^{-iH(t_i)\Delta t}|\psi(t_i)\rangle = U_i|\psi(t_i)\rangle$$

siendo $U_i = e^{-iH(t_i)\Delta t}$ una matriz unitaria.



Si discretizamos la evolución en $p = \frac{T}{\Delta t}$ pasos, podemos escribir:

$$
|\psi(t_1)\rangle = U_0|\psi(t_0)\rangle \\
|\psi(t_2)\rangle = U_1|\psi(t_1)\rangle \\
\cdots \\
|\psi(t_{p})\rangle = U_{p-1}|\psi(t_{p-1})\rangle
$$


De esta forma, el estado final del sistema será:

$$
|\psi(t_{p})\rangle = U_{p-1}U_{p-2}\cdots U_{1}U_{0}|\psi(t_0)\rangle = \left(\prod_{m=p-1}^{0} U_m\right) |\psi(t_0)\rangle =
$$

El Hamiltoniano usado en computación adiabática es:

$$H(t) = A(t)H_0 + B(t)H_c$$

por lo que resulta:

$$
|\psi(T)\rangle = \left(\prod_{m=0}^{p-1} e^{i\Delta t(A'(t_m)H_0 + B'(t_m)H_c)}\right) |\psi(0)\rangle
$$

donde $A'(t_m) = -A(t_{p-1-m})$, $B'(t_m) = -B(t_{p-1-m})$, $|\psi(T)\rangle = |\psi(t_{p})\rangle$ y $|\psi(0)\rangle = |\psi(t_{0})\rangle$

En general, dadas dos matrices $A$ y $B$, la igualdad $e^{A+B} = e^Ae^B$ solo se da si $A$ y $B$ _conmutan_, es decir si $[A,B] = AB-BA = 0$.

Si $\Delta t$ es lo suficientemente pequeño, la formula de Lie-Trotter permite escribir:

$$
e^{i\Delta t(A'(t_m)H_0 + B'(t_m)H_c)} \approx e^{i\Delta tA'(t_m)H_0} e^{i\Delta tB'(t_m)H_c}
$$

Por lo tanto, el estado final del sistema se puede escribir:

$$
|\psi_f\rangle = |\psi(T)\rangle = \left(\prod_{m=0}^{p-1} e^{i\Delta tA'(t_m)H_0} e^{i\Delta tB'(t_m)H_c}\right) |\psi(0)\rangle =
\prod_{m=0}^{p-1} U_0(t_m)U_c(t_m)|\psi_0\rangle
$$

siendo las matrices unitarias:

$$
U_0(t_m) = e^{i\Delta tA'(t_m)H_0}\\
U_c(t_m) = e^{i\Delta tB'(t_m)H_c}
$$

Este procedimiento se denomina _trotterization_ y es la base de algoritmos como el QAOA.




---



---



---




<a name="qaoa"></a>
# **Quantum Approximate Optimization Algorithm (QAOA)**

Algoritmo híbrido basado en la discretización de la evolución adiabática para resolver problemas Ising.

Como ya hemos visto, el estado final del sistema despues de la evolución es:

$$
|\psi_f\rangle = |\psi(T)\rangle = \left(\prod_{m=0}^{p-1} e^{i\Delta tA'(t_m)H_0} e^{i\Delta tB'(t_m)H_c}\right) |\psi(0)\rangle =
\prod_{m=0}^{p-1} U_0(t_m)U_c(t_m)|\psi_0\rangle
$$

siendo las matrices unitarias:

$$
U_0(t_m) = e^{i\Delta tA'(t_m)H_0} = e^{i\beta_mH_0} = U_0(\beta_m)\\
U_c(t_m) = e^{i\Delta tB'(t_m)H_c} = e^{i\gamma_mH_c} = U_c(\gamma_m)
$$
con $m=0,\ldots,p-1$ y $p \ge 1$.

QAOA intenta elegir unos valores de $\beta_m$ y $\gamma_m$ que aproximen el estado $|\psi_f\rangle$ al estado base del Hamiltoniano de coste $H_c$, es decir, que minimizen $\langle\psi_f|H_c|\psi_f\rangle$.

El algoritmo QAOA parte de un Hamiltoniano de coste $H_c$ que define el problema y procede como sigue:

1. Se elige un valor de $p\ge 1$ y dos listas de valores $\boldsymbol{\beta} = (\beta_0,\ldots,\beta_{p-1})$ y $\boldsymbol{\gamma} = (\gamma_0,\ldots,\gamma_{p-1})$
1. Como $H_0$ se suele usar el _mixing Hamiltonian_ y como estado inicial se suele usar la superposición completa
  $$H_0 = -\sum_{i=0}^{n-1} X_i\quad
  |\psi_0\rangle = |+\rangle^{\otimes n}$$
2. La QPU aplica las puertas $U_0(\beta_m)U_c(\gamma_m)$, $m=0,\ldots,p-1$, y se obtiene un nuevo estado $|\psi_p\rangle$
3. En la CPU se usa un algoritmo de optimización para actualizar los parámetros $\boldsymbol{\beta}$ y $\boldsymbol{\gamma}$ intentando minimizar el valor esperado $\langle\psi_p|H_c|\psi_p\rangle$
4. Se vuelve al paso 2 hasta obtener los valores óptimos $\boldsymbol{\beta}^\ast$ y $\boldsymbol{\gamma}^\ast$
5. Usando esos valores, se obtiene el estado que minimiza la solución

El proceso de optimización de los parámetros puede ser un simple gradiente descendente, en el que cada parámetro se actualiza en la dirección que conduzca al mayor descenso del valor esperado, u otros método de optimización más sofisticados.

### Construcción de $U_0(\beta)$ y $U_c(\gamma)$

#### Puertas para $U_0(\beta)$

Si tenemos $n$ cúbits, y dado que las matrices $X$ conmutan tenemos:

$$U_0(\beta) = e^{i\beta H_0} = e^{-i\beta\sum_{i=0}^{n-1} X_i} = \prod_{i=0}^{n-1}e^{-i\beta X_i}$$

Y recordando que la puerta de rotación $R_x(\theta) = e^{-i\theta X/2}$, se tiene:

$$U_0(\beta) = \prod_{i=0}^{n-1} R_x^{(i)}(2\beta)$$

La puerta $R_x^{(i)}$ indica que se aplica una $R_x$ al cúbit $i$ y la identidad al resto $\implies$ el producto equivale a aplicar una puerta $R_x(2\beta)$ a cada uno de los cúbits.

#### Puertas para $U_c(\gamma)$

El Hamiltoniano de coste de un problema Ising es:


$$H_c = -\sum_{i,j = 0}^{n-1} J_{ij}Z_iZ_j -\sum_{i = 0}^{n-1} h_i Z_i$$

siendo $J_{ij}$ y $h_i$ números reales.

Ya que las matricez $Z$ conmutan, podemos escribir

$$
U_c(\gamma) = e^{i\gamma H_c} = e^{-i\gamma(\sum_{i,j = 0}^{n-1} J_{ij}Z_iZ_j +\sum_{i = 0}^{n-1} h_i Z_i)} =
\prod_{i,j = 0}^{n-1}e^{-i\gamma J_{ij}Z_iZ_j}\prod_{i=0}^{n-1}e^{-i\gamma h_i Z_i}
$$

El segundo término se puede implementar con puertas $R_z(\theta) = e^{-i\theta Z/2}$.


Para el primer término, $Z_iZ_j$ representa aplicar una puerta $Z$ a los cúbits $i$ y $j$ y la identidad al resto:

$$
Z_iZ_j = I\otimes\ldots\otimes Z\otimes\ldots\otimes Z \otimes\ldots I = Z_i\otimes Z_j
$$

Y el término $e^{-i\gamma J_{ij}Z_i\otimes Z_j}$ corresponde a una [puerta Ising ZZ](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZZGate.html#qiskit.circuit.library.RZZGate) entre los cúbits $i$ y $j$:

$$
R_{ZZ}(\theta) = \exp\left(-i\frac{\theta}{2}(Z\otimes Z)\right) =
\exp{\left(-i\frac{\theta}{2}\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix} \right)}=
\begin{bmatrix}
e^{-i\theta/2} & 0 & 0 & 0\\
0 & e^{i\theta/2} & 0 & 0\\
0 & 0 & e^{i\theta/2} & 0\\
0 & 0 & 0 & e^{-i\theta/2}\\
\end{bmatrix}
$$

Con lo que queda:

$$
U_c(\gamma) = \prod_{i,j = 0}^{n-1} R_{ZZ}^{(i,j)}(2\gamma J_{ij})\prod_{i=0}^{n-1}R_z^{(i)}(2\gamma h_i)
$$


### Ejemplo

Obtener el circuito QAOA para un paso ($p$=1) del Hamiltoniano Ising:

$$
H_c = 3Z_2Z_0 - Z_2Z_1 + 2Z_0
$$

Necesitamos 3 cúbits, y el estado que queremos conseguir será:

$$
U_0(\beta)U_c(\gamma) |+\rangle^{\otimes 3}
$$

Con

$$
U_0(\beta) = \prod_{i=0}^{2} R_x^{(i)}(2\beta) = R_x^{(0)}(2\beta)\otimes R_x^{(1)}(2\beta)\otimes R_x^{(2)}(2\beta)
$$

$$
U_c(\gamma) = \prod_{i,j = 0}^{2} R_{ZZ}^{(i,j)}(2\gamma J_{ij})\prod_{i=0}^{2}R_z^{(i)}(2\gamma h_i) = (R_{ZZ}^{(2,0)}(6\gamma)\otimes R_{ZZ}^{(2,1)}(-2\gamma))(R_z^{(0)}(4\gamma))
$$

El circuito sería:

In [ ]:
from qiskit.circuit import Parameter

n = 3

beta = Parameter('β')
gamma = Parameter('γ')

qc = QuantumCircuit(n)

# Estado superpuesto
qc.h(range(3))

# Puertas U_c
qc.rzz(6*gamma, 2, 0)
qc.rzz(-2*gamma, 2, 1)
qc.rz(4*gamma, 0)

# puertas U_0
for i in range(3):
    qc.rx(2*beta, i)

qc.draw('mpl')

Podemos usar la implementación de QAOA de Qiskit para obtener este mismo circuito

In [ ]:
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp

# Expresamos el problema como un operador Hamiltoniano
Hc = SparsePauliOp.from_list([("ZIZ", 3), ("ZZI", -1), ("IIZ", 2)])

# Creamos el ansatz QAOA
ansatz = QAOAAnsatz(Hc, reps=1)

ansatz.decompose(reps=2).draw("mpl")

La puerta $U_2$ es una rotación de 1 cúbit generalizada, de la forma:

$$
U_2(\phi, \lambda) = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1          & -e^{i\lambda} \\
e^{i\phi} & e^{i(\phi + \lambda)}
\end{pmatrix}
$$

Así, una puerta $H$ es equivalente a:

$$
H = U_2(0, \pi) = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1          & -e^{i\pi} \\
e^{i0} & e^{i(0 + \pi)}
\end{pmatrix} =
\frac{1}{\sqrt{2}}\begin{pmatrix}
1          & 1 \\
1 & -1
\end{pmatrix}
$$

In [ ]:
from qiskit.quantum_info import Statevector

# Creamos una lista con todos los estados base para n cúbits
estados = [Statevector.from_int(i, dims=2**n) for i in range(2**n)]

for i in range(2**n):
  print('Estado ',i, 'valor esperado=',estados[i].expectation_value(Hc).real)

Usamos la implementación de [QAOA](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.QAOA.html) del paquete [qiskit_algorithms](https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.html) para obtener el mínimo

In [ ]:
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit.result import QuasiDistribution

# Sampler: nueva primitiva de Qiskit que obtiene
# muestras del circuito para estimar el valor esperado.
sampler = Sampler()

algorithm_globals.random_seed = 10598

optimizer = COBYLA()

qaoa = QAOA(sampler, optimizer)

# Obtiene el mínimo autovalor del Hamiltoniano
result = qaoa.compute_minimum_eigenvalue(Hc)

print(result,'\n')

bistring = result.best_measurement['bitstring']
valor = result.best_measurement['value']
print("Estado que minimiza Hc = {}".format(bistring))
print("Valor esperado <{}|Hc|{}> = {}".format(bistring,bistring,valor.real))

# Nota: el warning se debe a que QAOA no está migrado a la V2 de Sampler



---



---



---



<a name="cuadraticos"></a>

## **Programas cuadráticos con restricciones cuadráticas**

La librería de optimización de Qiskit incorpora una librería para resolver problemas QCQP con variables binarias y/o enteras.

**Ejemplo**

Minimiza $y=10x_0+20x_1+30 x_2 +  4 x_0x_1 + 2 x_0x_2 + 6 x_1x_2 + 4 x_1^2  + 2 x_2^2$

con las siguientes restricciones:

  - $x_0\in\{0,1\}$
  - $x_1, x_2 \in \mathbb{Z}, -1\le x_1 \le 1, -2\le x_2 \le 3$

Podemos expresar el problema de la siguiente forma:ç

Se puede escribir como:
$$
\begin{align}
\text{minimizar} &&x^T Q x + c^T x &&\\
&& && \\
\text{sujeto a} &&x_0\in\{0,1\}, x_1, x_2 \in \mathbb{Z}  &&\\
&& -1\le x_1 \le 1 \\
&& -2\le x_2 \le 3 \\
\end{align}
$$

Se puede probar fácilmente que $Q$ y $c$ en este ejemplo valen:

$$
\begin{aligned}
Q  &= \begin{bmatrix}0 & 1 & 2 \\ 3 & 4 & 5 \\ 0 & 1 & 2 \end{bmatrix}\\[10pt]
c^T &= \begin{bmatrix}10&20&30\end{bmatrix}
\end{aligned}
$$

In [ ]:
from qiskit_optimization import QuadraticProgram

# Definimos el problema cuadrático
qprog = QuadraticProgram('Ejemplo')

# Fijamos el tipo de las variables y sus restricciones
qprog.binary_var(name = 'x_0')
qprog.integer_var(name = 'x_1', lowerbound = -1, upperbound = 1)
qprog.integer_var(name = 'x_2', lowerbound = -2, upperbound = 3)

# Matriz Q y vector c
Q = [[0,1,2],[3,4,5],[0,1,2]]
c = [10,20,30]

# Exprersamos el problema como una minimización
qprog.minimize(quadratic = Q, linear = c)

print(qprog.prettyprint())

Este problema puede ser resuelto mediante QAOA usando el método [`MinimumEigenOptimizer`](https://qiskit-community.github.io/qiskit-optimization/stubs/qiskit_optimization.algorithms.MinimumEigenOptimizer.html) que se encarga de convertir el programa cuadrático a un circuito y de realizar la optimización.

A este método hay que pasarle un objeto de una clase que implemente la interfaz [`MinimumEigenSolver`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.MinimumEigensolver.html), como puede ser un objeto [`VQE`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.VQE.html) o [`QAOA`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.QAOA.html).

In [ ]:
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import ADAM
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer

qaoa = QAOA(Sampler(), ADAM(), initial_point=[0.0, 0.0])

# Creamos el optimizador
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa)

# Usamos ese optimizador sobre el problema anterior
result = eigen_optimizer.solve(qprog)

print(result)

Podemos comprobar que el resultado es correcto usando un minimizador clásico.

In [ ]:
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

np_solver = NumPyMinimumEigensolver()
np_optimizer = MinimumEigenOptimizer(min_eigen_solver = np_solver)

result = np_optimizer.solve(qprog)
print(result)



---



---



---



<a name="maxcut"></a>

## **MAX-CUT como QUBO**

Ya vimos que el MAX-CUT se puede expresar como un problema QUBO, con una función de coste:
$$
C(\textbf{x}) = \sum_{i,j = 0}^{n-1} w_{ij} x_i (1-x_j)
$$
siendo todas las $x_i$ variables binarias.

Esta función de coste se puede reescribir como:

$$
\begin{align}
\sum_{i,j=0}^{n-1} w_{ij} x_i (1-x_j) &= \sum_{i,j=0}^{n-1} w_{ij} x_i - w_{ij}x_i x_j  \\
&= \sum_{i=0}^{n-1} \left( \sum_{j=0}^{n-1} w_{ij} \right) x_i - \sum_{i,j = 0}^{n-1} w_{ij}x_i x_j \\
&= c^T x + x^T Q x, \\
\end{align}
$$
siendo $Q$ y $c$:
$$
Q_{ij} = -w_{ij} \qquad c_i = \sum_{j=1}^n w_{ij}.
$$

In [ ]:
import networkx as nx  # Librería para manejar grafos

# Ejemplo de grafo con 6 nodos
nnodes = 6
G = nx.Graph()
# Añade nodos y aristas
G.add_nodes_from(np.arange(0,nnodes,1))
edges = [(0,1,2.0),(0,2,3.0),(0,3,2.0),(0,4,4.0),(0,5,1.0),(1,2,4.0),(1,3,1.0),
         (1,4,1.0),(1,5,3.0),(2,4,2.0),(2,5,3.0),(3,4,5.0),(3,5,1.0)]
G.add_weighted_edges_from(edges)

# Mostramos el grafo
layout = nx.random_layout(G,seed=10)
colors = ['red', 'green', 'lightblue', 'yellow', 'magenta', 'gray']
nx.draw_networkx(G, layout, node_color=colors)
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=labels);

In [ ]:
print("Matriz de adyacencias del grafo")
print(nx.adjacency_matrix(G).toarray())

Vamos a obtener por fuerza bruta el coste de todos los cortes.

In [ ]:
# Función de coste de Maxcut
def maxcut_coste(graph, bitstring):
    """
    Computa la función de coste del Maxcut para un grafo y un corte representado por un string de bits
    Args:
        graph: El grafo networkx
        bitstring: str
                   Un string con valores 0 o 1 especificando un corte en el grafo
    Returns:
        coste: float
               El coste del corte
    """
    # Matriz de adyacencias del grafo
    weight_matrix = nx.adjacency_matrix(graph).toarray()
    coste = 0
    for i, j in graph.edges():
        # Si los vértices están en conjuntos distintos suma el peso de la arista
        if bitstring[i] != bitstring[j]:
            coste += weight_matrix[i,j]

    return coste

In [ ]:
# Obtenemos por fuerza bruta los valores de todos los posibles cortes
num_vars = G.number_of_nodes()

#Creamos una lista de bitstrings con todos los posibles cortes
bitstrings = ['{:b}'.format(i).rjust(num_vars, '0') for i in range(2**num_vars)]
print(bitstrings)

costes = list()
for bitstring in bitstrings:
    costes.append(maxcut_coste(G, bitstring))
print(costes)

# Ordenamos los costes y los bitstrings en orden creciente de coste
costes, bitstrings = zip(*sorted(zip(costes, bitstrings)))

In [ ]:
print("Corte de mayor coste = ",bitstrings[len(bitstrings)-1], ", coste = ", costes[len(bitstrings)-1])


In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
# Muestra una gráfica con los valores
bar_plot = go.Bar(x = bitstrings, y = costes, marker=dict(color=costes, colorscale = 'plasma', colorbar=dict(title='Cut Value')))
fig = go.Figure(data=bar_plot, layout = dict(xaxis=dict(type = 'category'), width = 1500, height = 600))
fig.show()



---



Resolvemos el problema de optimización con QAOA

In [ ]:
# Nº de nodos del grafo y matriz de pesos
nnodes = len(G.nodes())
weight_matrix = nx.adjacency_matrix(G).toarray()

# Matriz Q
Q = -weight_matrix

# Vector c
c = np.zeros(nnodes)
for i in range(nnodes):
    for j in range(nnodes):
        c[i] += weight_matrix[i,j]

# Defino el problema
qubo_maxcut = QuadraticProgram('Maxcut como problema cuadrático')

# Variables binarias del problema
for i in range(nnodes):
    nombre = 'x_'+str(i)
    qubo_maxcut.binary_var(name = nombre)

# Especificalo como un problema de maximizacion e imprimelo
qubo_maxcut.maximize(quadratic = Q, linear = c)
print(qubo_maxcut.prettyprint())

In [ ]:
# Usamos el optimizador anterior sobre este problema
result = eigen_optimizer.solve(qubo_maxcut)
print(result)

#### Referencias

  - Farhi, E., Goldstone, J., & Gutmann, S. (2014). A quantum approximate optimization algorithm. arXiv preprint [arXiv:1411.4028](https://arxiv.org/abs/1411.4028)
  - Ejemplo definiendo el circuito: https://learning.quantum.ibm.com/tutorial/quantum-approximate-optimization-algorithm



---



---



---

